# **Instaling necessary libraries**

In [1]:
!pip install transformers opendatasets langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.2 MB/s eta 0:00:00


# **Importing necessary libraries**

In [19]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
import numpy as np
import torch
import opendatasets as od
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# **Downloded IMDB-MovieReview from Kaggle**

In [3]:
od.download('https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nishantsingh96
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


100%|██████████| 25.7M/25.7M [00:00<00:00, 719MB/s]

In [4]:
Data=pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv',chunksize=1000)
Data=next(Data)
Data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# **Loded model and tokenizer for training via LoRA**

In [5]:
base_model=AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
classifier=pipeline("text-classification", model="distilbert-base-uncased", return_tensors=True)
tokenizer=classifier.tokenizer

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
tokenized=tokenizer(Data['review'].tolist(),padding="max_length",truncation=True,max_length=256)

In [8]:
tokenized['labels']=Data['sentiment'].replace({'positive':1, 'negative':0})

/tmp/ipython-input-8-3881689977.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tokenized['labels']=Data['sentiment'].replace({'positive':1, 'negative':0})


In [9]:
dataset=Dataset.from_dict(tokenized)
dataset=dataset.train_test_split(test_size=0.1)

# **Defining LoRA config**

In [10]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS,
    lora_dropout=0.1,
    bias="none"
)
model=get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


In [11]:
training_args=TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

# **Training model subparameters**

In [12]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
)

/tmp/ipython-input-12-4266177553.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
trainer.train()

Step,Training Loss
10,0.687800
20,0.688900
30,0.709800
40,0.683500
50,0.655300
60,0.670600
70,0.663300
80,0.650800
90,0.649200
100,0.654800


TrainOutput(global_step=114, training_loss=0.6684517881326508, metrics={'train_runtime': 30.4739, 'train_samples_per_second': 59.067, 'train_steps_per_second': 3.741, 'total_flos': 121265466163200.0, 'train_loss': 0.6684517881326508, 'epoch': 2.0})

# **Load and save the fine tuned model for future purpose**

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
trainer.save_model("/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model")
tokenizer.save_pretrained("/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model")

('/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model/tokenizer_config.json',
 '/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model/special_tokens_map.json',
 '/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model/vocab.txt',
 '/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model/added_tokens.json',
 '/content/drive/MyDrive/Public/DistilBert/lora-finetuned-model/tokenizer.json')

# **Re initialize model weights and tokenizer**

In [17]:
model_path="/content/drive/MyDrive/Public /DistilBert/lora-finetuned-model"

tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): lora.Linear(
              (base_layer): Linear(in_features=768, out_features=768, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=768, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=768, bi

# **Define nodes for LangGraph**
* Inference_node
* confidence_check_node
* fallback_nodes

In [25]:
class State(TypedDict):
    text: str
    prediction: str
    confidence: float


def inference_node(state: State) -> State:
    inputs=tokenizer(state["text"], return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    with torch.no_grad():
        outputs=model(**inputs)
        probs=torch.softmax(outputs.logits, dim=-1)
        confidence=torch.max(probs).item()
        label=torch.argmax(probs).item()
        prediction=["negative", "positive"][label]

        print(f"[InferenceNode] Predicted label: {prediction.capitalize()} | Confidence: {confidence:.0%}")

        return {
            "text": state["text"],
            "prediction": prediction,
            "confidence": confidence
        }


def confidence_check_node(state: State) -> State:
    if state["confidence"] < 0.7:
        print("[ConfidenceCheckNode] Confidence too low. Triggering fallback...")
    else:
        print("[ConfidenceCheckNode] Confidence acceptable.")
    return state

def should_fallback(state: State) -> str:
    return "fallback" if state["confidence"] < 0.7 else END


def fallback_node(state: State) -> State:
    print(f"[FallbackNode] Could you clarify your intent? Was this a {'negative' if state['prediction']=='negative' else 'positive'} review?")
    user_input=input("\nUser: ").strip().lower()

    if "yes" in user_input and "negative" in user_input:
        final_prediction="negative"
        print(f"Final Label: Negative (Corrected via user clarification)")
    elif "yes" in user_input and "positive" in user_input:
        final_prediction="positive"
        print(f"Final Label: Positive (Corrected via user clarification)")
    elif "no" in user_input:
        final_prediction="positive" if state['prediction'] == "negative" else "negative"
        print(f"Final Label: {final_prediction.capitalize()} (Corrected via user clarification)")
    else:
        final_prediction=state['prediction']
        print(f"Final Label: {final_prediction.capitalize()} (Keeping original)")

    return {
        "text": state["text"],
        "prediction": final_prediction,
        "confidence": state["confidence"]
    }

In [26]:
graph=StateGraph(State)

graph.add_node("inference", RunnableLambda(inference_node))
graph.add_node("confidence_check", RunnableLambda(confidence_check_node))
graph.add_node("fallback", RunnableLambda(fallback_node))

graph.set_entry_point("inference")
graph.add_edge("inference", "confidence_check")

graph.add_conditional_edges(
    "confidence_check",
    should_fallback,
    {
        "fallback": "fallback",
        END: END
    }
)

graph.add_edge("fallback", END)


app=graph.compile()


print('---'*30)
inp=input("Enter a review: ")
result=app.invoke({"text": inp, "prediction": "", "confidence": 0.0})
print('---'*30)

------------------------------------------------------------------------------------------
Enter a review: The movie was boring and time consuming
[InferenceNode] Predicted label: Negative | Confidence: 56%
[ConfidenceCheckNode] Confidence too low. Triggering fallback...
[FallbackNode] Could you clarify your intent? Was this a negative review?

User: yes it's negative
Final Label: Negative (Corrected via user clarification)
------------------------------------------------------------------------------------------
